#### Imports

In [1]:
from tifffile import TiffFile
import re
import pandas as pd
from tqdm import tqdm
import sys
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sys.path.append(r'C:\Users\97254\Desktop\git\FluoroVision\src')

In [3]:
from post_process.fluorophore_intensity import  get_bead_fluoro_intensity
from utils.common_utils import extract_frame_number

#### Functions

#### Inputs

In [4]:
video_path = r'C:\Users\97254\Desktop\Resources\Technion\exploratory_resaerach\data\AB3C.tif'
annotation_path = r'C:\Users\97254\Desktop\Resources\Technion\exploratory_resaerach\data\AB3C_gt_new_format.csv'

In [5]:
df = pd.read_csv(annotation_path)

In [6]:
df['frame_number'] = df['frame'].apply(extract_frame_number)

In [14]:
fluoro_intesity1_list, fluoro_intesity2_list = [], []
save = True
peak_th = 1000
with TiffFile(video_path) as tif:
      for i, page in tqdm(enumerate(tif.pages)):
        frame = page.asarray()
        frame_df = df.loc[df['frame_number'] == i+1]
        box_list = []
        for index, row in frame_df.iterrows():
            box_list.append([row['x_center'], row['y_center'], row['width'], row['height']])
        for j, box in enumerate(box_list):
            # if np.any((two_bead_peaks == (i,j)).all(axis=1)):
            #     print('debug')
            # else:
            #     continue
            if save:
                save_path = r'C:\Users\97254\Desktop\Resources\Technion\exploratory_resaerach\debug\fluoropgore_estimation\AB3C_peak_th\frame_{}_box_{}.png'.format(i, j)
                fluoro_intesity1, fluoro_intesity2 = get_bead_fluoro_intensity(frame, box, num_peaks=3, min_distance=10, peak_radius=3, peak_th=peak_th,save=save, save_path=save_path)
            else:
                fluoro_intesity1, fluoro_intesity2 = get_bead_fluoro_intensity(frame, box, num_peaks=3, min_distance=10, peak_radius=3,peak_th=peak_th,save=save, save_path=None)
            fluoro_intesity1_list.append(fluoro_intesity1)
            fluoro_intesity2_list.append(fluoro_intesity2)

0it [00:00, ?it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
1it [00:00,  1.31it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
2it [00:01,  1.32it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
3it [00:02,  1.32it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
4it [00:03,  1.31it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
5it [00:03,  1.29it/s]Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
6it [00:05,  1.07s/it]Clipping input data to the valid range for imshow with RGB data ([0..1] for floa